## Set Up

In [0]:
#Initialize TPU
%tensorflow_version 1.x
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [0]:
#Code to install necessary libraries and perform authorization
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#Mount google drive 
!mkdir -p drive 
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.19-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.19-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.19-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2

In [0]:
#Path
import os
os.chdir('/content/drive/musicProject')

#No need to clone Magenta git
#Already cloned

## Loading/Installing Magenta

In [0]:
#@test {"output": "ignore"}
%tensorflow_version 1.x

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files
import magenta.music as mm
import magenta
import tensorflow

print(magenta.__version__)
print(tensorflow.__version__)

TensorFlow 1.x selected.
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 144573 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Preparing to unpack .../libasound2-dev_1.1.3-5ubuntu0.5_amd64.deb ...
Unpacking libasound2-dev:amd64 (1.1.3-5ubuntu0.5) over (1.1.3-5ubuntu0.4) ...
Preparing to unpack .../libasound2_1.1.3-5ubuntu0.5_amd64.deb ...
Unpacking libasound2:amd64 (1.1.3-5ubuntu0.5) over (1.1.3-5ubuntu0.4) ...
Preparing to unpack .../libasound2-data_1.1.3-5ubuntu0.5_all.deb ...
Unpacking libasound2-data (1.1.3-5ubuntu0.5) over (1.1.3-5ubuntu0.4) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libasound2-data (1.1.3-5ubuntu0.5) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libasound2:amd64 (1.1.3-5ubuntu0.

## Creating Dataset (data processing 1)

Midi files will be converted to a dataset of note sequences.

In [0]:
#input_dir is path to trainingFile folder containing all downloaded midis
#output_file is path to output and name for noteSequences file

!python /content/drive/Colab_Notebooks/musicProject/magenta/magenta/scripts/convert_dir_to_note_sequences.py --input_dir /content/drive/Colab_Notebooks/musicProject/data/trainingFile --output_file /content/drive/Colab_Notebooks/musicProject/data/noteSequences.tfrecord --recursive

#not sure how to get it to run using multilines

## Create Sequence Examples for Training (data processing 2)

Sequence Examples created from above dataset will contain a sequence of inputs and a sequence of labels that represent a polyphonic sequence.

In [0]:
#input is the path to noteSequence.tfrecord file from above
#output_dir is path to output file

!python /content/drive/Colab_Notebooks/musicProject/magenta/magenta/models/polyphony_rnn/polyphony_rnn_create_dataset.py --input /content/drive/Colab_Notebooks/musicProject/data/noteSequences.tfrecord --output_dir /content/drive/Colab_Notebooks/musicProject/data/datasequence_examples --eval_ratio=0.10

#not sure how to get it to run using multilines

## Model Training/Eval

In [0]:
#run_dir is where to store checkpoints
#sequence_example_file is the generated file from above
#num_training_steps is number of training steps
#hparams is hyperparameters where you can specify batch size and rnnLayer size (will be using 3 layers with 256 units)

with strategy.scope():
    #trainingjob
    !python /content/drive/musicProject/magenta/magenta/models/polyphony_rnn/polyphony_rnn_train.py --run_dir /content/tmp/polyphonyTrainingCheckpoints2/logdir/run1 --sequence_example_file /content/drive/musicProject/data/sequence_examples/training_poly_tracks.tfrecord --hparams "batch_size=64,rnn_layer_sizes=[256,256,256]" --num_training_steps 5000
    #parallel evaluation job
    !python /content/drive/musicProject/magenta/magenta/models/polyphony_rnn/polyphony_rnn_train.py --run_dir /content/tmp/polyphonyTrainingCheckpoints2/logdir/run1 --sequence_example_file /content/drive/musicProject/data/sequence_examples/eval_poly_tracks.tfrecord --hparams "batch_size=64,rnn_layer_sizes=[256,256,256]" --num_eval_examples 5000 --eval

In [0]:
#Save model
#bundle_model is where to save your saved model and name

!python /content/drive/musicProject/magenta/magenta/models/polyphony_rnn/polyphony_rnn_generate.py --run_dir /content/tmp/polyphonyTrainingCheckpoints2/logdir/run1 --hparams "batch_size=64,rnn_layer_sizes=[256,256,256]" --bundle_file /content/drive/musicProject/trainedModels/trainedPolyphony_rnn.mag --save_generator_bundle

## Polyphonic Track Generation

In [0]:
#Use saved model to generate track
#for me trainedPolyphony_rnn4584.mag is the most recent save model and 4584 was the most recent training step

!python /content/drive/musicProject/magenta/magenta/models/polyphony_rnn/polyphony_rnn_generate.py --bundle_file /content/drive/musicProject/trainedModels/trainedPolyphony_rnn4584.mag --output_dir /content/drive/musicProject/samples --num_outputs 15 --num_steps 800 --primer_midi /content/drive/musicProject/data/midiArtists/midiRandomParts/dh1.mid --condition_on_primer true --inject_primer_during_generation true

2020-04-18 02:32:54.028138: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-18 02:32:54.071664: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-18 02:32:54.071726: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (65d1ba63fb79): /proc/driver/nvidia/version does not exist
2020-04-18 02:32:54.111836: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-18 02:32:54.113044: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x76e36c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-18 02:32:54.113111: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
INFO:tensorflow:Restoring parameters from /tmp/tmpgsak5f5r/mod